## Predicting Book Ratings on GoodReads using Linear Regression Models

In this section, I will develop a linear regression model to predict the "rating" value for GoodReads books.

For this particular model, I will take advantage of the fact that each book in the original dataframe has an extensive list of genres and some of them have also received multiple awards. Before reducing their quantity, there were 980 unique genres and 5831 unique awards.

Once these features have been extracted and converted to numerical variables in previous kernels, I expect them to help me build a more robust model to predict the "rating".

The next steps in the process will be to import the cleaned dataframe, reduce, adapt and try to select the appropriate features for modeling, build the model, check the metrics, and make improvements if possible.

In [390]:
import pandas as pd
import numpy as np

### Import Dataframe

Importing clean dataframe
- Cleaning and wrangling done in "1. goodreads_data_CLEANING_AND_WRANGLING.ipynb"

In [391]:
gr_data = pd.read_csv('./2. Clean_df/gr_data_CLEAN.csv')
gr_data = gr_data.drop(['Unnamed: 0'],axis=1)

In [392]:
display(gr_data.head(), gr_data.shape)

,title,series,author,description,language,pages,isbn,book_format,publish_year,first_publish_year,...,West Australian Young Readers' Book Award (WAYRBA) for Older Readers,West Australian Young Readers' Book Award (WAYRBA) for Younger Readers,William Allen White Children's Book Award,William C. Morris YA Debut Award Nominee,Women's Prize for Fiction Nominee,Women's Prize for Fiction Nominee for Longlist,World Fantasy Award Nominee for Best Novel,World Fantasy Award for Best Novel,Zilveren Griffel,الجائزة العالمية للرواية العربية (أي باف) / International Prize for Arabic Fiction (IPAF) Nominee
0,The Hunger Games,The Hunger Games,Suzanne Collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,200-300,9.78044E+12,Hardcover,2008,2008,...,1,0,0,0,0,0,0,0,0,0
1,Harry Potter and the Order of the Phoenix,Harry Potter,J.K. Rowling,There is a door at the end of a silent corrido...,English,700-800,9.78044E+12,Paperback,2004,2003,...,0,0,0,0,0,0,0,0,0,0
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,English,200-300,1E+13,Paperback,2006,1960,...,0,0,0,0,0,0,0,0,0,0
3,Pride and Prejudice,Single Book,Jane Austen,Alternate cover edition of ISBN 9780679783268S...,English,100-200,1E+13,Paperback,2000,2013,...,0,0,0,0,0,0,0,0,0,0
4,Twilight,The Twilight Saga,Stephenie Meyer,About three things I was absolutely positive.\...,English,400-500,9.78032E+12,Paperback,2006,2005,...,1,0,0,0,0,0,0,0,0,0


(48655, 862)

### Preprocessing data and features for modeling

#### Drop unnecesary columns

In [393]:
gr_data= gr_data.drop(['isbn','description'],axis=1)

In [394]:
gr_data.shape

(48655, 860)

#### NaN

In [395]:
gr_data.isna().sum().sum()

0

#### Types

In [396]:
gr_data.iloc[:, :18].dtypes

title                   object
series                  object
author                  object
language                object
pages                   object
book_format             object
publish_year             int64
first_publish_year       int64
publisher               object
liked_perc             float64
bbe_score              float64
bbe_votes              float64
log_num_ratings        float64
5_stars_num_ratings      int64
4_stars_num_ratings      int64
3_stars_num_ratings      int64
2_stars_num_ratings      int64
1_star_num_ratings       int64
dtype: object

In [398]:
numerical_columns = gr_data.select_dtypes(include=np.number).columns.tolist()
print(f"Numerical columns: {len(numerical_columns)}")

categorical_columns = gr_data.select_dtypes(include='object').columns.tolist()
print(f"Categorical columns: {len(categorical_columns)}")

Numerical columns: 853
Categorical columns: 7


### Dealing with type of 'genres' and 'awards' multiple columns

After extracting their names from their respective lists, I created two dataframes in which each unique genre and award was converted into a single column. If the original list for a book (row) contained that genre or award, the corresponding value in that record was assigned a 1; otherwise, it was assigned a 0.

The only exceptions are the columns that hold aggregated numbers for the binned awards in 'Unique Award (1)', 'Uncommon Award (2,3)', 'Singular Award (4-8)', and 'Frequently Awarded Literary Prizes (9-15)'. In order to reduce the thousands of unique awards, I grouped them according to their frequency.

Since the binned award columns should be treated as numerical data and the 1-0 column values as categorical, I will import those dataframes and assign the appropriate data types.

In [400]:
# Importing dataframes
award_df = pd.read_csv('./0. Drafts/award_df.csv')
award_df = award_df.drop(['Unnamed: 0'],axis=1)
award_df = award_df.drop(['index_gr_data'],axis=1)

genres_df = pd.read_csv('./0. Drafts/genres_clean_dataset.csv')
genres_df = genres_df.drop(['Unnamed: 0'],axis=1)
genres_df = genres_df.drop(['index_gr_data'],axis=1)

In [401]:
# As all values in genre_df are 0 or 1, I'm appending them into binary_columns list
binary_columns = [column for column in genres_df.columns]
len(binary_columns)

609

In [402]:
# Extracting binary column names and appending to previous list.
for column in award_df.columns:
    unique_values = award_df[column].unique()
    if set(unique_values) != set([0, 1]):
        continue
        #non_binary_columns.append(column)
    elif set(unique_values) == set([0, 1]):
        binary_columns.append(column)

In [403]:
# New length
len(binary_columns)

844

In [405]:
# Loop through columns names in gr_data; if listed in binary_columns converted to 'object' type
for column in gr_data.columns:
    if column in binary_columns:
        gr_data[column] = gr_data[column].astype('object')

In [406]:
numerical_columns = gr_data.select_dtypes(include=np.number).columns.tolist()
print(f"Numerical columns: {len(numerical_columns)}")

categorical_columns = gr_data.select_dtypes(include='object').columns.tolist()
print(f"Categorical columns: {len(categorical_columns)}")

Numerical columns: 16
Categorical columns: 844


#### Numerical Data

In [407]:
numerical = gr_data.select_dtypes(include=np.number)
numerical.shape
#categorical = gr_data.select_dtypes(include=['object'])

(48655, 16)

##### Correlation-Association amongst features

Correlation Matrix:

In [408]:
# Correlation matrix for numerical values (854 columns)
corr_matrix = numerical.corr()

- As there are so many columns, displaying data where correlations > 0.6 or < -0,6 :

In [409]:
# Indices of non-NaN values in the correlation matrix
indices = [(row, col) for row in corr_matrix.index for col in corr_matrix.columns if not pd.isna(corr_matrix.loc[row, col])]

# New df with the row and column's names, and correlation number
df = pd.DataFrame([(idx[0], idx[1], corr_matrix.loc[idx]) for idx in indices if (corr_matrix.loc[idx] > 0.6 or corr_matrix.loc[idx] < -0.6) and idx[0] != idx[1]], columns=['row name', 'col name', 'corr number'])

# Drop duplicate rows where row name is greater than col name (diagonal)
df = df[df['row name'] < df['col name']]

# Sort df by the correlation number in descending order
df = df.sort_values(by='corr number', ascending=False)

# Sorted df
display(df, len(df))


,row name,col name,corr number
11,3_stars_num_ratings,4_stars_num_ratings,0.957836
16,2_stars_num_ratings,3_stars_num_ratings,0.956072
0,bbe_score,bbe_votes,0.939596
21,1_star_num_ratings,2_stars_num_ratings,0.933824
6,4_stars_num_ratings,5_stars_num_ratings,0.931859
15,2_stars_num_ratings,4_stars_num_ratings,0.856281
10,3_stars_num_ratings,5_stars_num_ratings,0.831870
20,1_star_num_ratings,3_stars_num_ratings,0.820076
14,2_stars_num_ratings,5_stars_num_ratings,0.726403
19,1_star_num_ratings,4_stars_num_ratings,0.711987


11

 - Check counts from paired highly-correlated variables to choose which one to eliminate

In [11]:
#display(numerical['Hugo Award for Best Novel '].value_counts(),numerical['hugo awards'].value_counts())

0    48593
1       62
Name: Hugo Award for Best Novel , dtype: int64

0    48620
1       35
Name: hugo awards, dtype: int64

In [12]:
#display(numerical['love inspired'].value_counts(),numerical['love inspired historical'].value_counts())

0    48645
1       10
Name: love inspired, dtype: int64

0    48650
1        5
Name: love inspired historical, dtype: int64

In [410]:
# High correlated variables to drop
high_correlated = ['2_stars_num_ratings','3_stars_num_ratings','4_stars_num_ratings',
                   'bbe_votes']

In [13]:
# High correlated variables to drop
#high_correlated = ['2_stars_num_ratings','3_stars_num_ratings','4_stars_num_ratings',
#                   'bbe_votes','hugo awards','booze','love inspired historical']

In [411]:
numerical.shape

(48655, 16)

In [412]:
# Drop high correlated variables
numerical = numerical.drop(high_correlated, axis=1)

# New shape
numerical.shape

(48655, 12)

correlation matrix can differ depending on whether the data is scaled or not. This is because correlation measures the linear relationship between two variables, and the scale of the variables can affect the strength of their relationship.

For example, if one variable has a much larger scale than another variable, it may dominate the correlation analysis and obscure any relationships between the other variables. Scaling the variables to have the same range of values can help to mitigate this problem and provide a more accurate representation of the correlations between the variables.

generally recommended to scale the numerical features before applying PCA to ensure that they are on a similar scale and have equal weight in the analysis. This is because PCA is sensitive to the scale of the variables, and features with larger scales can dominate the analysis. 

#### Categorical Data

In [413]:
categorical = gr_data.select_dtypes(include=['object'])
categorical.shape

(48655, 844)

In [414]:
# Unique values
for col in categorical.columns:
    print(f"Unique values for {col}: {categorical[col].nunique()}")

Unique values for title: 46328
Unique values for series: 4405
Unique values for author: 21720
Unique values for language: 92
Unique values for pages: 12
Unique values for book_format: 10
Unique values for publisher: 4
Unique values for fantasy: 2
Unique values for science fiction: 2
Unique values for dystopia: 2
Unique values for young adult: 2
Unique values for fiction: 2
Unique values for action: 2
Unique values for adventure: 2
Unique values for romance: 2
Unique values for magic: 2
Unique values for children: 2
Unique values for audiobook: 2
Unique values for school: 2
Unique values for classics: 2
Unique values for historical fiction: 2
Unique values for novels: 2
Unique values for literature: 2
Unique values for historical: 2
Unique values for historical romance: 2
Unique values for adult: 2
Unique values for urban fantasy: 2
Unique values for paranormal: 2
Unique values for paranormal romance: 2
Unique values for vampires: 2
Unique values for holocaust: 2
Unique values for war: 

- Reducing unique values

Author

In [415]:
categorical['author']

0        Suzanne Collins
1           J.K. Rowling
2             Harper Lee
3            Jane Austen
4        Stephenie Meyer
              ...       
48650      Sherry Gammon
48651      Emma Michaels
48652     Kim Richardson
48653        Tom Pollack
48654       Misty Moncur
Name: author, Length: 48655, dtype: object

Reducing number of authors by grouping them into 'Top 1000', 'Top 1001-10000' and 'General Authors' according to the average 'rating' for authors that have published more than 1 book (there are a lot of authors with just 1 book published and a 5 rating).

In [416]:
# Reducing number of unique authors: TOP 1000 (average rating for authors with >1 book published)

# Count the number of books for each author
author_counts = gr_data['author'].value_counts()

# Filter the resulting series to only include authors with a count greater than 1
filtered_authors = author_counts[author_counts > 1].index

# Filter the original dataframe to only include titles with authors in the filtered series
filtered_gr_data = gr_data[gr_data['author'].isin(filtered_authors)]

# Group by author and calculate the mean rating for each author
author_avg_rating = filtered_gr_data.groupby('author')['rating'].mean()

# Sort the resulting series in descending order based on the average rating and select the top 100 authors
top_1000_most_rated_authors = author_avg_rating.sort_values(ascending=False).head(1000)

# Convert the resulting series to a dataframe and reset the index
top_1000_most_rated_authors_df = top_1000_most_rated_authors.to_frame().reset_index()

display(len(top_1000_most_rated_authors_df))

top_1000_most_rated_authors_df.head(30)

1000

,author,rating
0,James Goodman,5.00
1,"Doug ""Hollywood"" Davis",5.00
2,Natasha Lukin,5.00
3,Rita Pam Tarachi,5.00
4,Granthana Sinha,5.00
5,Nadine May,5.00
6,Bernard Cenney,5.00
7,Luke A.M. Brown,5.00
8,Susan Davis,5.00
9,Rodolfo Martin Vitangcol,5.00


In [417]:
# Reducing number of unique authors: TOP 1001-10.000

# Count the number of books for each author
author_counts = gr_data['author'].value_counts()

# Filter the resulting series to only include authors with a count greater than 1
filtered_authors = author_counts[author_counts > 1].index

# Filter the original dataframe to only include titles with authors in the filtered series
filtered_gr_data = gr_data[gr_data['author'].isin(filtered_authors)]

# Group by author and calculate the mean rating for each author
author_avg_rating = filtered_gr_data.groupby('author')['rating'].mean()

# Sort the resulting series in descending order based on the average rating and select the top 1001 to 10000 authors
top_1001_to_10000_most_rated_authors = author_avg_rating.sort_values(ascending=False).iloc[1000:10000]

# Convert the resulting series to a dataframe and reset the index
top_1001_to_10000_most_rated_authors_df = top_1001_to_10000_most_rated_authors.to_frame().reset_index()

display(len(top_1001_to_10000_most_rated_authors_df))

top_1001_to_10000_most_rated_authors_df.head(30)


6180

,author,rating
0,Adrienne Rich,4.276667
1,E.F. Benson,4.276667
2,Lexie Xu,4.276667
3,Anna Zaires,4.276667
4,Mandi Beck,4.276667
5,Arnold Lobel,4.276667
6,Lois Walfrid Johnson,4.276667
7,Amy Clipston,4.276667
8,J.M. Darhower,4.276000
9,Mike Mignola,4.276000


In [418]:
# Other authors

# Count the number of books for each author
author_counts = gr_data['author'].value_counts()

# Filter the resulting series to only include authors with a count equal to 1
single_authors = author_counts[author_counts == 1].index

# Filter the original dataframe to only include titles with authors not in the top rated authors lists or with only 1 book
general_authors_data = gr_data[~gr_data['author'].isin(top_1000_most_rated_authors.index) & ~gr_data['author'].isin(top_1000_most_rated_authors.index) & gr_data['author'].isin(single_authors)]
#general_authors_data = gr_data[~gr_data['author'].isin(top_1000_most_rated_authors.index) & ~gr_data['author'].isin(top_1001_to_10000_most_rated_authors.index) & ~gr_data['author'].isin(author_counts[author_counts == 1].index)]

# Group by author and calculate the mean rating for each author
general_authors_avg_rating = general_authors_data.groupby('author')['rating'].mean()

# Convert the resulting series to a dataframe and reset the index
general_authors_avg_rating_df = general_authors_avg_rating.to_frame().reset_index()

display(len(general_authors_avg_rating_df))

general_authors_avg_rating_df.head(30)


14540

,author,rating
0,50 Cent,4.15
1,A. Bates,3.54
2,A. Cort Sinnes,4.50
3,A. Helwa,4.63
4,A. Merritt,3.33
5,A. Roger Ekirch,3.68
6,A. Scott Berg,4.00
7,A. Teeuw,4.06
8,A. Ubaidillah Alias,3.94
9,A. White,4.93


In [419]:
# Check result

# Count the number of unique authors in the original dataframe
num_unique_authors = len(gr_data['author'].unique())

# Count the number of unique authors in new authors dataframes
num_top_1000_authors = len(top_1000_most_rated_authors_df)
num_top_1001_to_10000_authors = len(top_1001_to_10000_most_rated_authors_df)
num_general_authors = len(general_authors_avg_rating_df)

# Check if the sum of the number of new dataframes is equal to unique authors in the original dataframe
if num_top_1000_authors + num_top_1001_to_10000_authors + num_general_authors == num_unique_authors:
    print("The number of unique authors in the three new dataframes is equal to the number of unique authors in the original gr_data dataframe.")
else:
    print("The number of unique authors in the three new dataframes is not equal to the number of unique authors in the original gr_data dataframe.")

The number of unique authors in the three new dataframes is equal to the number of unique authors in the original gr_data dataframe.


In [420]:
# Replace the author values in categorical['author'] with the appropriate category
categorical['author'] = np.where(categorical['author'].isin(top_1000_most_rated_authors_df['author']), 'top_1000_most_rated_authors',
                    np.where(categorical['author'].isin(top_1001_to_10000_most_rated_authors_df['author']), 'top_1001_to_10000_most_rated_authors',
                    np.where(categorical['author'].isin(general_authors_avg_rating_df['author']), 'general_authors',
                    categorical['author'])))

categorical.head(10)

,title,series,author,language,pages,book_format,publisher,fantasy,science fiction,dystopia,...,West Australian Young Readers' Book Award (WAYRBA) for Older Readers,West Australian Young Readers' Book Award (WAYRBA) for Younger Readers,William Allen White Children's Book Award,William C. Morris YA Debut Award Nominee,Women's Prize for Fiction Nominee,Women's Prize for Fiction Nominee for Longlist,World Fantasy Award Nominee for Best Novel,World Fantasy Award for Best Novel,Zilveren Griffel,الجائزة العالمية للرواية العربية (أي باف) / International Prize for Arabic Fiction (IPAF) Nominee
0,The Hunger Games,The Hunger Games,top_1001_to_10000_most_rated_authors,English,200-300,Hardcover,Top Publishers,1,1,1,...,1,0,0,0,0,0,0,0,0,0
1,Harry Potter and the Order of the Phoenix,Harry Potter,top_1000_most_rated_authors,English,700-800,Paperback,Top Publishers,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,To Kill a Mockingbird,To Kill a Mockingbird,top_1001_to_10000_most_rated_authors,English,200-300,Paperback,Top Publishers,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pride and Prejudice,Single Book,top_1001_to_10000_most_rated_authors,English,100-200,Paperback,Top Publishers,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Twilight,The Twilight Saga,top_1001_to_10000_most_rated_authors,English,400-500,Paperback,Top Publishers,1,1,0,...,1,0,0,0,0,0,0,0,0,0
5,The Book Thief,Single Book,top_1001_to_10000_most_rated_authors,English,400-500,Hardcover,Top Publishers,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Animal Farm,Single Book,top_1001_to_10000_most_rated_authors,English,<100,Paperback,Top Publishers,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,The Chronicles of Narnia,Single Book,top_1001_to_10000_most_rated_authors,English,600-700,Paperback,Top Publishers,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,The Lord of the Rings-3,top_1001_to_10000_most_rated_authors,English,900-1000,Paperback,Top Publishers,1,1,0,...,0,0,0,0,0,0,0,0,0,0
9,Gone with the Wind,Single Book,top_1000_most_rated_authors,English,900-1000,Paperback,Top Publishers,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [421]:
categorical['author'].unique()

array(['top_1001_to_10000_most_rated_authors',
       'top_1000_most_rated_authors', 'general_authors'], dtype=object)

Series

In [422]:
categorical['series'].nunique()

4405

Same idea as before, this time grouping series into 'Top 500 rated series' and 'Other Series' and keeping 'Single Books' for those books not included in any literary saga. And again, criteria to bin them is according to the average 'rating' for each series.

In [423]:
# Filter the original dataframe to only include titles that are not 'Single Book'
series_data = gr_data[gr_data['series'] != 'Single Book']

# Group by series and calculate the mean rating for each series
series_avg_rating = series_data.groupby('series')['rating'].mean()

# Sort the resulting series in descending order based on the average rating and select the top 500 series
top_500_most_rated_series = series_avg_rating.sort_values(ascending=False).head(500)

# Filter the original dataframe to only include titles with series not in the top rated series lists or with empty series
other_series_data = series_data[~series_data['series'].isin(top_500_most_rated_series.index) & (series_data['series'] != '')]

# Group by series and calculate the mean rating for each series
other_series_avg_rating = other_series_data.groupby('series')['rating'].mean()

# Convert the resulting series to a dataframe and reset the index
top_500_most_rated_series_df = top_500_most_rated_series.to_frame().reset_index()
other_series_avg_rating_df = other_series_avg_rating.to_frame().reset_index()


In [424]:
# Replace series values in categorical['series'] with the appropriate category
categorical['series'] = np.where(categorical['series'] == 'Single Book', 'Single Book',
                    np.where(categorical['series'].isin(top_500_most_rated_series_df['series']), 'Top 500 rated series',
                    np.where(categorical['series'].isin(other_series_avg_rating_df['series']), 'Other series',
                    categorical['series'])))


In [425]:
categorical['series'].unique()

array(['Other series', 'Top 500 rated series', 'Single Book'],
      dtype=object)

Title

In [426]:
categorical['title'].nunique()

46328

- As there are a lot of unique authors, I have decided to drop that column

In [427]:
categorical = categorical.drop(['title'],axis=1)

Language

In [428]:
categorical['language'].value_counts()

English                                  43415
Arabic                                     894
Spanish                                    632
French                                     562
German                                     501
                                         ...  
Iranian (Other)                              1
Aromanian; Arumanian; Macedo-Romanian        1
gu                                           1
Faroese                                      1
Afrikaans                                    1
Name: language, Length: 92, dtype: int64

In [429]:
# Count the unique values in the 'language' column
language_counts = categorical['language'].value_counts()

# Top 5 most common languages
top_languages = language_counts.index[:5]
top_languages

Index(['English', 'Arabic', 'Spanish', 'French', 'German'], dtype='object')

All languages not included in top_languages are going to be grouped as 'Other languages'

In [430]:
# Replace all the other languages with 'Other languages'
categorical.loc[~categorical['language'].isin(top_languages), 'language'] = 'Other languages'

In [431]:
categorical['language'].unique()

array(['English', 'French', 'German', 'Other languages', 'Arabic',
       'Spanish'], dtype=object)

In [432]:
categorical['language'].nunique()

6

In [433]:
# Unique values
for col in categorical.columns:
    print(f"Unique values for {col}: {categorical[col].nunique()}")

Unique values for series: 3
Unique values for author: 3
Unique values for language: 6
Unique values for pages: 12
Unique values for book_format: 10
Unique values for publisher: 4
Unique values for fantasy: 2
Unique values for science fiction: 2
Unique values for dystopia: 2
Unique values for young adult: 2
Unique values for fiction: 2
Unique values for action: 2
Unique values for adventure: 2
Unique values for romance: 2
Unique values for magic: 2
Unique values for children: 2
Unique values for audiobook: 2
Unique values for school: 2
Unique values for classics: 2
Unique values for historical fiction: 2
Unique values for novels: 2
Unique values for literature: 2
Unique values for historical: 2
Unique values for historical romance: 2
Unique values for adult: 2
Unique values for urban fantasy: 2
Unique values for paranormal: 2
Unique values for paranormal romance: 2
Unique values for vampires: 2
Unique values for holocaust: 2
Unique values for war: 2
Unique values for politics: 2
Unique 

#### Concatenate numerical and categorical processed data

In [434]:
gr_data_for_modelling = pd.concat([categorical,numerical],axis=1)

In [435]:
display(gr_data_for_modelling.head(3),gr_data_for_modelling.shape)

,series,author,language,pages,book_format,publisher,fantasy,science fiction,dystopia,young adult,...,liked_perc,bbe_score,log_num_ratings,5_stars_num_ratings,1_star_num_ratings,rating,Unique Award (1),"Uncommon Award (2,3)",Singular Award (4-8),Frequently Awarded Literary Prizes (9-15)
0,Other series,top_1001_to_10000_most_rated_authors,English,200-300,Hardcover,Top Publishers,1,1,1,1,...,96.0,6.476225,6.804602,3444695,93557,4.33,4,5,3,15
1,Top 500 rated series,top_1000_most_rated_authors,English,700-800,Paperback,Top Publishers,1,1,0,1,...,98.0,6.420324,6.399262,1593642,14526,4.50,2,0,1,3
2,Other series,top_1001_to_10000_most_rated_authors,English,200-300,Paperback,Top Publishers,0,0,0,1,...,95.0,6.355911,6.653316,2363896,80794,4.28,0,0,0,2


(48655, 855)

In [436]:
# NaN values
gr_data_for_modelling.isna().sum().sum()

0

In [437]:
# Saving data processed for modelling
gr_data_for_modelling.to_csv('./2. Clean_df/gr_data_MODELLING.csv')

## Linear Regression Model

In [438]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#import scipy.stats as stats

In [439]:
gr_data = pd.read_csv('./2. Clean_df/gr_data_MODELLING.csv')
gr_data = gr_data.drop(['Unnamed: 0'],axis=1)
gr_data.shape

(48655, 855)

In [440]:
gr_data.head()

,series,author,language,pages,book_format,publisher,fantasy,science fiction,dystopia,young adult,...,liked_perc,bbe_score,log_num_ratings,5_stars_num_ratings,1_star_num_ratings,rating,Unique Award (1),"Uncommon Award (2,3)",Singular Award (4-8),Frequently Awarded Literary Prizes (9-15)
0,Other series,top_1001_to_10000_most_rated_authors,English,200-300,Hardcover,Top Publishers,1,1,1,1,...,96.0,6.476225,6.804602,3444695,93557,4.33,4,5,3,15
1,Top 500 rated series,top_1000_most_rated_authors,English,700-800,Paperback,Top Publishers,1,1,0,1,...,98.0,6.420324,6.399262,1593642,14526,4.50,2,0,1,3
2,Other series,top_1001_to_10000_most_rated_authors,English,200-300,Paperback,Top Publishers,0,0,0,1,...,95.0,6.355911,6.653316,2363896,80794,4.28,0,0,0,2
3,Single Book,top_1001_to_10000_most_rated_authors,English,100-200,Paperback,Top Publishers,0,0,0,0,...,94.0,6.297348,6.476867,1617567,76770,4.26,0,0,0,0
4,Other series,top_1001_to_10000_most_rated_authors,English,400-500,Paperback,Top Publishers,1,1,0,1,...,78.0,6.164189,6.695877,1751460,548674,3.60,3,5,4,6


#### X-y split

In [441]:
# Target = 'rating'
y = gr_data['rating']
X = gr_data.drop(['rating'],axis=1)

#### Train-Test Split

In [442]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [443]:
#### Split both Train and Test into numerical and categoricals

In [444]:
# X_train
X_train_num = X_train.select_dtypes(include=np.number)
X_train_cat = X_train.select_dtypes(include=['object'])

# X_test
X_test_num = X_test.select_dtypes(include=np.number)
X_test_cat = X_test.select_dtypes(include=['object'])

#### Numerical Transformations

- Fit only with numericals_train

In [445]:
scaler = StandardScaler().fit(X_train_num)

- Transform both numericals_train and numericals_test

In [446]:
X_train_num_scaled = scaler.transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

In [447]:
# Convert X_train_num_scaled and X_test_num_scaled to dataframes
X_train_num_scaled_df = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns)
X_test_num_scaled_df = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)

#### Categorical Encoding

Fit only with categoricals_train

In [448]:
encoder = OneHotEncoder().fit(X_train_cat)

Encode both categoricals_train and numericals_test

In [449]:
X_train_cat_encoded = encoder.transform(X_train_cat)
X_test_cat_encoded = encoder.transform(X_test_cat)

In [450]:
# Convert X_train_cat_encoded and X_test_cat_encoded to dataframes
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded.toarray(), columns=encoder.get_feature_names_out())
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded.toarray(), columns=encoder.get_feature_names_out())

#### Combine numericals_train and categoricals_train into train_processed

In [451]:
train_processed = pd.concat([X_train_num_scaled_df, X_train_cat_encoded_df], axis=1)

#### Combine numericals_test and categoricals_test into test_processed

In [452]:
test_processed = pd.concat([X_test_num_scaled_df, X_test_cat_encoded_df], axis=1)

#### Define Linear Regression Model

- Fit model on train_processed

In [453]:
lr = LinearRegression().fit(train_processed, y_train)

In [454]:
# Predictions on training data
y_train_pred = lr.predict(train_processed)

print("First 5 predictions on the training data:", y_train_pred[:5])

# Evaluation metrics
r2 = r2_score(y_train, y_train_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
mse = mean_squared_error(y_train, y_train_pred)
mae = mean_absolute_error(y_train, y_train_pred)

print("\nR-squared score on the training data:", round(r2, 2))
print("Root mean squared error on the training data:", round(rmse, 2))
print("Mean squared error on the training data:", round(mse, 2))
print("Mean absolute error on the training data:", round(mae, 2))

First 5 predictions on the training data: [3.73948669 4.16273499 3.79444885 4.09135437 3.90409851]

R-squared score on the training data: 0.45
Root mean squared error on the training data: 0.26
Mean squared error on the training data: 0.07
Mean absolute error on the training data: 0.16


- Predictions on test_processed and scores

In [455]:
# Predictions on test data
y_test_pred = lr.predict(test_processed)

print("Actual data:", y_test[:6])
print("\nPredictions on the TEST data:", y_test_pred[:6])

# Evaluation metrics on test data
r2_test = r2_score(y_test, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print("\nR-squared score on the test data:", round(r2_test, 2))
print("Root mean squared error on the test data:", round(rmse_test, 2))
print("Mean squared error on the test data:", round(mse_test, 2))
print("Mean absolute error on the test data:", round(mae_test, 2))

Actual data: 13219    4.05
28726    4.00
46900    3.86
13985    4.51
40148    4.10
27223    3.97
Name: rating, dtype: float64

Predictions on the TEST data: [4.00071716 4.00016785 3.89833069 4.48484802 3.92564392 3.77653503]

R-squared score on the test data: 0.42
Root mean squared error on the test data: 0.27
Mean squared error on the test data: 0.07
Mean absolute error on the test data: 0.17


In [456]:
gr_data['rating'].describe()

count    48655.000000
mean         4.015514
std          0.347446
min          0.000000
25%          3.820000
50%          4.020000
75%          4.220000
max          5.000000
Name: rating, dtype: float64

After trying to use 'genres' and 'awards' features both as numerical or categorical, the model is not performing well on the test data:

An R-squared score of 0.42 indicates that the model explains 42% of the variance in the target variable, which is moderately good.

The root mean squared error (RMSE) on the test data is 0.27 ('rating' std = 0.34), which means that on average, the model's predictions are off by 0.27 units from the actual values.

The mean squared error (MSE) on the test data is 0.07, which is the average squared difference between the predicted and actual values.

The mean absolute error (MAE) on the test data is 0.17, which is the average absolute difference between the predicted and actual values.

In terms of pros and cons, here are some for PCA:

Pros:

Can help reduce the number of features in high-dimensional datasets, making it easier to fit a model and reducing the risk of overfitting.
Can reveal patterns and relationships in the data that may not be apparent in the original feature space.
Can help with data visualization by reducing the dimensionality of the data to 2 or 3 dimensions.
Cons:

PCA assumes that the principal components represent the most important patterns in the data, which may not always be true.
It can be difficult to interpret the meaning of the principal components, especially if there are many of them.
Depending on the amount of variance you want to retain, PCA can result in a loss of information.

In order to check if I can get better results, I'm going to reduce features applying PCA. To see the an

In [ ]:
# Fit PCA on training data
pca = PCA(n_components=100)
pca.fit(train_processed)

# Get the explained variance ratios
var_ratios = pca.explained_variance_ratio_

# Calculate the cumulative sum of variance explained
cum_var_explained = np.cumsum(var_ratios)

# Plot the cumulative explained variance vs number of components
plt.plot(range(1, len(cum_var_explained)+1), cum_var_explained)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()


In [ ]:
from sklearn.decomposition import PCA

# Fit PCA on training data
pca = PCA(n_components=100)
pca.fit(train_processed)

# Transform training and test data using PCA
X_train_pca = pca.transform(train_processed)
X_test_pca = pca.transform(test_processed)

# Fit linear regression on transformed data
lr_pca = LinearRegression().fit(X_train_pca, y_train)

# Predict on test set
y_pred_pca = lr_pca.predict(X_test_pca)

# Calculate R-squared score
r2_pca = r2_score(y_test, y_pred_pca)
print("R-squared score (PCA transformed):", r2_pca)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Fit PCA on training data
pca = PCA(n_components=100)
pca.fit(train_processed)

# Transform training and test data using PCA
X_train_pca = pca.transform(train_processed)
X_test_pca = pca.transform(test_processed)

# Fit random forest regressor on transformed data
rf = RandomForestRegressor(n_estimators=100, max_depth=10).fit(X_train_pca, y_train)